In [6]:
import duckdb

In [5]:
!pip install --quiet jupysql duckdb-engine

In [7]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:default:

In [36]:
%%sql

create or replace table opensea_events as (
    select
        *
    from
        read_json_auto('../messages/*.jsonl')
);
select
    *
from
    opensea_messages;

,event,payload,ref,topic
0,phx_reply,"{'response': '{}', 'status': 'ok', 'event_type...",0.0,collection:*
1,phx_reply,"{'response': '{}', 'status': 'ok', 'event_type...",0.0,phoenix
2,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
3,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
4,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
...,...,...,...,...
751,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
752,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
753,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
754,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*


In [37]:
%%sql

create or replace view collection_events as (
    select
        *
    from
        opensea_events
    where
        topic = 'collection:*'
        and event != 'phx_reply'
);
select
    *
from
    collection_events;

,event,payload,ref,topic
0,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
1,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
2,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
3,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
4,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
...,...,...,...,...
853,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
854,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
855,item_metadata_updated,"{'response': None, 'status': None, 'event_type...",NaN,collection:*
856,item_transferred,"{'response': None, 'status': None, 'event_type...",NaN,collection:*


In [70]:
%%sql

create or replace table stg_collection_events as (
    select
        event
        , payload->>'sent_at' as sent_at
        , payload->>'event_type' as event_type
        , strptime(payload->'payload'->>'event_timestamp', '%Y-%m-%dT%H:%M:%S.%f%z') as event_timestamp_ts
        , payload->'payload'->'collection'->>'slug' as collection_slug
        , payload->'payload'->'item'->>'permalink' as item_permalink
        , payload->'payload'->'item'->>'nft_id' as item_nft_id
        , payload->'payload'->'item'->'chain'->>'name' as item_chain_name
        , payload->'payload' as payload
    from
        collection_events
);
select
    *
from
    stg_collection_events;

,event,sent_at,event_type,event_timestamp_ts,collection_slug,item_permalink,item_nft_id,item_chain_name,payload
0,item_transferred,2023-10-07T20:32:05.267859+00:00,item_transferred,2023-10-07 22:32:00+02:00,flexiblereturntoken-5,https://testnets.opensea.io/assets/sepolia/0x0...,sepolia/0x09547e68ce13fdecb5bf52fd17379fffa97c...,sepolia,"{""chain"":""sepolia"",""collection"":{""slug"":""flexi..."
1,item_metadata_updated,2023-10-07T20:32:05.420618+00:00,item_metadata_updated,NaT,flexiblereturntoken-5,https://testnets.opensea.io/assets/sepolia/0x0...,sepolia/0x09547e68ce13fdecb5bf52fd17379fffa97c...,sepolia,"{""chain"":null,""collection"":{""slug"":""flexiblere..."
2,item_transferred,2023-10-07T20:32:05.239814+00:00,item_transferred,2023-10-07 22:32:00+02:00,gabbyprompt-3,https://testnets.opensea.io/assets/sepolia/0x2...,sepolia/0x2c2b60634793e57c895fc91129c863e3127e...,sepolia,"{""chain"":""sepolia"",""collection"":{""slug"":""gabby..."
3,item_transferred,2023-10-07T20:32:05.285113+00:00,item_transferred,2023-10-07 22:32:00+02:00,flexiblereturntoken-5,https://testnets.opensea.io/assets/sepolia/0x0...,sepolia/0x09547e68ce13fdecb5bf52fd17379fffa97c...,sepolia,"{""chain"":""sepolia"",""collection"":{""slug"":""flexi..."
4,item_metadata_updated,2023-10-07T20:32:05.392220+00:00,item_metadata_updated,NaT,flexiblereturntoken-5,https://testnets.opensea.io/assets/sepolia/0x0...,sepolia/0x09547e68ce13fdecb5bf52fd17379fffa97c...,sepolia,"{""chain"":null,""collection"":{""slug"":""flexiblere..."
...,...,...,...,...,...,...,...,...,...
853,item_metadata_updated,2023-10-07T20:15:43.045540+00:00,item_metadata_updated,NaT,lll-12,https://testnets.opensea.io/assets/goerli/0xeb...,goerli/0xebffe1d60ec936b23f633b243af8016e94edf...,goerli,"{""chain"":null,""collection"":{""slug"":""lll-12""},""..."
854,item_metadata_updated,2023-10-07T20:15:43.106667+00:00,item_metadata_updated,NaT,lll-12,https://testnets.opensea.io/assets/goerli/0xeb...,goerli/0xebffe1d60ec936b23f633b243af8016e94edf...,goerli,"{""chain"":null,""collection"":{""slug"":""lll-12""},""..."
855,item_metadata_updated,2023-10-07T20:15:43.044782+00:00,item_metadata_updated,NaT,lll-12,https://testnets.opensea.io/assets/goerli/0xeb...,goerli/0xebffe1d60ec936b23f633b243af8016e94edf...,goerli,"{""chain"":null,""collection"":{""slug"":""lll-12""},""..."
856,item_transferred,2023-10-07T20:15:43.049098+00:00,item_transferred,2023-10-07 22:15:36+02:00,lll-12,https://testnets.opensea.io/assets/goerli/0xeb...,goerli/0xebffe1d60ec936b23f633b243af8016e94edf...,goerli,"{""chain"":""goerli"",""collection"":{""slug"":""lll-12..."


In [67]:
%%sql

select
    collection_slug
    , count(*) as event_count
from
    stg_collection_events
group by
    collection_slug
order by
    event_count desc
limit 10;

,collection_slug,event_count
0,flexiblereturntoken-5,456
1,lll-12,202
2,vera-uj9hinocnp,24
3,gnomes-collective-2,20
4,bird-22,14
5,gabbyexplore-1,11
6,unidentified-contract-syi4yutfpm,10
7,gabbyexperience-2,8
8,demask-creator-19,7
9,badgetoken-eo1x1icfxe,6


In [71]:
%%sql

-- Top 10 item chains
select
    item_chain_name
    , count(*) as event_count
from
    stg_collection_events
group by
    item_chain_name
order by
    event_count desc
limit 10;

,item_chain_name,event_count
0,sepolia,496
1,goerli,254
2,mumbai,56
3,optimism_goerli,34
4,bsctestnet,6
5,avalanche_fuji,5
6,base_goerli,4
7,arbitrum_goerli,3
